In [0]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [0]:
# 사용되는 GPU 상태 보여주는 코드
import tensorflow as tf
from tensorflow.python.client import device_lib

tf.test.gpu_device_name()
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17093504356585333123, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14828404428544861581
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 15815470637660546102
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14892338381
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4771973201577832715
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [1]:
# 구글 코랩에 구글 드라이브 연동하는 코드
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# 모델 테스트용 데이터 준비: tf_Model 이 입력으로 1000*1000 이미지를 받을 때 사용했던 거
import numpy as np
import tensorflow as tf
import cv2

# 모델이 만들어지는지 테스트용 데이터 생성, 20개
image = []
for i in range(1, 21):
    image.append(np.zeros((1000, 1000, 1)))

image = np.array(image)
    
# 라벨 데이터 준비
t_list = np.zeros((len(image),5))

val = 0
cnt = 0
for i in range(len(image)):
    t_list[i][val] = 1
    
    cnt+=1
    if cnt == 4:
        cnt=0
        val+=1

# 이미지와 라벨 모양 출력
print(image.shape)
print(t_list.shape)


(20, 1000, 1000, 1)
(20, 5)


In [0]:
# 모델 차원 테스트
# 아래에서 모델을 생성한 후 위에있는 테스트 데이터를 사용해서 
# 전방향 진행에 대한 차원의 변화 확인
x_b=image
y_b=t_list

print('x_b shape: ', x_b.shape)
print('y_b shape: ', y_b.shape)

model.print_proc(x_b, y_b)

x_b shape:  (20, 1000, 1000, 1)
y_b shape:  (20, 5)
(20, 450, 450, 16)
(20, 200, 200, 32)
(20, 90, 90, 64)
(20, 518400)
(20, 125)
(20, 2)
(20, 5)


In [0]:
# 직접 만든 모델
# 학습이 잘 이루어지지 않음
import tensorflow as tf
import numpy as np

class tf_Model:
    def __init__(self):
        # 세션 준비
        tf.reset_default_graph()
        self.sess = tf.Session()
        
    def var_set(self, params):
        x_h = params[0]
        x_w = params[1]
        x_c = params[2]
        
        out_n = params[3]
        
        f_s1 = params[4]
        f_s2 = params[5]
        f_s3 = params[6]
        
        f_n1 = params[7]
        f_n2 = params[8]
        f_n3 = params[9]
        
        a_n1 = params[10]
        a_n2 = params[11]
        a_n3 = params[12]
        
        # 입력 데이터와 라벨을 담을 변수 준비
        self.x = tf.placeholder(tf.float32,shape=[None,x_h,x_w,x_c], name='x')
        self.y_label = tf.placeholder(tf.float32,shape=[None,out_n], name='y')
        
        # 가중치 및 바이어스 초기화
        self.weight_conv1 = tf.Variable(tf.truncated_normal([f_s1,f_s1,x_c,f_n1],stddev=0.1), name='W1')
        self.bias_conv1 = tf.Variable(tf.constant(0.1,shape=[f_n1]), name='b1')
        
        #2. 두번째 계층의 가중치와 바이어스(컨블루션 계층)
        self.weight_conv2 = tf.Variable(tf.truncated_normal([f_s2,f_s2,f_n1,f_n2],stddev=0.1), name='W2')
        self.bias_conv2 = tf.Variable(tf.constant(0.1,shape=[f_n2]), name='b2')
        
        #3. 세번째 계층의 가중치와 바이어스(컨블루션 계층)
        self.weight_conv3 = tf.Variable(tf.truncated_normal([f_s3,f_s3,f_n2,f_n3],stddev=0.1), name='W3')
        self.bias_conv3 = tf.Variable(tf.constant(0.1,shape=[f_n3]), name='b3')
        
        #4. 네번째 계층의 가중치와 바이어스(완전연결 계층)
        self.weight_affine1 = tf.Variable(tf.truncated_normal([a_n1,a_n2], stddev=0.1), name='W4')
        self.bias_affine1 = tf.Variable(tf.truncated_normal([a_n2]), name='b4')
        
        
        #5. 다섯번째 계층의 가중치와 바이어스(완전연결 계층)
        self.weight_affine2 = tf.Variable(tf.truncated_normal([a_n2,a_n3], stddev=0.1), name='W5')
        self.bias_affine2 = tf.Variable(tf.constant(0.1,shape=[a_n3]), name='b5')
        
        #6.  여섯번째 계층의 가중치와 바이어스(완전연결 계층)
        self.weight_affine3 = tf.Variable(tf.truncated_normal([a_n3,out_n], stddev=0.1), name='W6')
        self.bias_affine3 = tf.Variable(tf.constant(0.1,shape=[out_n]), name='b6')
        
        #5. 드롭아웃 기준 설정
        self.dropout_ratio = tf.placeholder(tf.float32, name='dropout_ratio')
        
        
        #1. 첫번째 계층(컨볼루션-활성화-폴링)
        self.conv1      = tf.nn.conv2d(self.x,self.weight_conv1,strides=[1,1,1,1], padding='SAME')
        self.conv1_add  = tf.add(self.conv1, self.bias_conv1)
        self.b_n1       = tf.layers.batch_normalization(self.conv1_add)
        self.actv1      = tf.nn.softplus(self.b_n1)
        self.pool1      = tf.nn.max_pool(self.actv1,ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')
        
        #2. 두번째 계층(컨볼루션-활성화-폴링)
        self.conv2      = tf.nn.conv2d(self.pool1,self.weight_conv2,strides=[1,1,1,1], padding='SAME')
        self.conv2_add  = tf.add(self.conv2, self.bias_conv2)
        self.b_n2       = tf.layers.batch_normalization(self.conv2_add)
        self.actv2      = tf.nn.softplus(self.b_n2)
        self.pool2      = tf.nn.max_pool(self.actv2,ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')
        
        #3. 세번째 계층(컨볼루션-활성화-폴링)
        self.conv3      = tf.nn.conv2d(self.pool2,self.weight_conv3,strides=[1,1,1,1], padding='SAME')
        self.conv3_add  = tf.add(self.conv3, self.bias_conv3)
        self.b_n3       = tf.layers.batch_normalization(self.conv3_add)
        self.actv3      = tf.nn.softplus(self.b_n3)
        self.pool3      = tf.nn.max_pool(self.actv3,ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')
        
        #4. 네번째 계층 수행 전 데이터 변환(각 데이터당 2차원데이터 -> 1차원데이터 변환)
        self.flat = tf.reshape(self.pool3,[-1,a_n1])
        
        #5. 네번째 계층(가중치와 행렬곱-활성화-드롭아웃처리)
        self.affine1 = tf.add(tf.matmul(self.flat,self.weight_affine1), self.bias_affine1, name='affine1')
        self.b_n4  = tf.layers.batch_normalization(self.affine1)
        self.actv4 = tf.nn.softplus(self.b_n4)
        self.drop1 = tf.nn.dropout(self.actv4,self.dropout_ratio)
        
        #6. 다섯번째 계층(가중치와 행렬곱-활성화-드롭아웃처리)
        self.affine2 = tf.add(tf.matmul(self.drop1,self.weight_affine2), self.bias_affine2, name='affine2')
        self.b_n5  = tf.layers.batch_normalization(self.affine2)
        self.actv5 = tf.nn.softplus(self.b_n5)
        self.drop2 = tf.nn.dropout(self.actv5,self.dropout_ratio)
        
        #7. 여섯번째 계층(가중치와 행렬곱-활성화)
        self.affine3 = tf.add(tf.matmul(self.drop2,self.weight_affine3), self.bias_affine3)
        self.softmax = tf.nn.softmax(self.affine3, name='soft_max')
        
        # 오차 계산
        self.cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.affine3, labels=self.y_label, name='cross_entropy')
        self.cost = tf.reduce_mean(self.cross_entropy)
        
        # 가중치 수정
        self.train_step = tf.train.AdamOptimizer(0.001).minimize(self.cost)
        
        # 정확도 측정
        self.correct_prediction = tf.equal(tf.argmax(self.softmax,1), tf.argmax(self.y_label,1), name='corr')
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32), name='acc')
                
    def model_make(self, params):
        '''
        여기서 텐서플로우로 인공 신경망 모델을 만든다
        매개변수로 주는 리스트 순서
        [x_h, x_w, x_c, out_n, f_s1, f_s2, .. , f_n1, f_n2, .. , a_n1, a_n2, .. ]
        
        x_h, x_w, x_c: 입력하는 데이터의 높이, 너비, 차원
        
        out_n     : 출력 계층의 노드 갯수
        
        f_s[숫자] : 컨볼루션 필터의 크기
        f_n[숫자] : 컨볼루션 계층의 필터 갯수
        
        a_n[숫자] : Affine 계층의 노드 갯수
        
        '''
        self.var_set(params)
        self.sess.run(tf.global_variables_initializer())
            
    def model_save(self, path):
        saver = tf.train.Saver()
        save_path = saver.save(self.sess, path)
        print("SavePath: %s"%(save_path))
        
    def model_load(self, path):
        saver = tf.train.Saver()
        saver.restore(self.sess, path)
        
    def model_train(self, x_b, y_b):
        self.sess.run(self.train_step,feed_dict={self.x: x_b, self.y_label: y_b, self.dropout_ratio: 0.5})
                        
    def get_relu(self, x_b, y_b):
        res = self.sess.run(self.actv5,feed_dict={self.x: x_b, self.y_label: y_b, self.dropout_ratio: 1.0})
        return res
        
    def get_softmax(self, x_b, y_b):
        res = self.sess.run(self.softmax,feed_dict={self.x: x_b, self.y_label: y_b, self.dropout_ratio: 1.0})
        return res
    
    def get_cross_entropy(self, x_b, y_b):
        res = self.sess.run(self.cross_entropy,feed_dict={self.x: x_b, self.y_label: y_b, self.dropout_ratio: 1.0})
        return res
    
    def get_correct_prediction(self, x_b, y_b):
        res = self.sess.run(self.correct_prediction,feed_dict={self.x: x_b, self.y_label: y_b, self.dropout_ratio: 1.0})
        return res
    
    def get_accuracy(self, x_b, y_b):
        res = self.sess.run(self.accuracy,feed_dict={self.x: x_b, self.y_label: y_b, self.dropout_ratio: 1.0})
        return res
    
    def get_xy(self, x_b):
        res = self.sess.run(self.affine2,feed_dict={self.x: x_b, self.dropout_ratio: 1.0})
        return res
    
    def print_proc(self, x_b, y_b):
        res = self.sess.run(self.pool1,feed_dict={self.x: x_b, self.dropout_ratio: 0.5})
        print(np.array(res).shape)
        res = self.sess.run(self.pool2,feed_dict={self.x: x_b, self.dropout_ratio: 0.5})
        print(np.array(res).shape)
        res = self.sess.run(self.pool3,feed_dict={self.x: x_b, self.dropout_ratio: 0.5})
        print(np.array(res).shape)
        res = self.sess.run(self.flat,feed_dict={self.x: x_b, self.dropout_ratio: 0.5})
        print(np.array(res).shape)
        res = self.sess.run(self.drop1,feed_dict={self.x: x_b, self.dropout_ratio: 0.5})
        print(np.array(res).shape)
        res = self.sess.run(self.drop2,feed_dict={self.x: x_b, self.dropout_ratio: 0.5})
        print(np.array(res).shape)
        res = self.sess.run(self.affine3,feed_dict={self.x: x_b, self.dropout_ratio: 0.5})
        print(np.array(res).shape)
        
    def print_accs(self, x_b, y_b):
        res1 = self.sess.run(self.cross_entropy,feed_dict={self.x: x_b, self.y_label: y_b, self.dropout_ratio: 1.0})
        res2 = self.sess.run(self.accuracy,feed_dict={self.x: x_b, self.y_label: y_b, self.dropout_ratio: 1.0})
        print(res1, res2)
        


In [0]:
# tf_Model 모델의 파라미터 준비

# 입력 데이터의 크기 지정
x_h = 224
x_w = 224
x_c = 3

# 출력 갯수 지정
out_n = 5

# 필터 크기 지정
f_s1 = 7
f_s2 = 5
f_s3 = 3

# 컨볼루션 층의 필터 갯수 지정
f_n1 = 16
f_n2 = 32
f_n3 = 64

# 출력 데이터 크기 지정
out_1 = x_h//2   # 첫 conv 계층 지난 후 크기
out_2 = out_1//2 # 두번째 conv 계층 지난 후 크기
out_3 = out_2//2 # 세번째 conv 계층 지난 후 크기

# 완전연결 계층의 노드 수 지정
a_n1 = out_3*out_3*f_n3
a_n2 = 125
a_n3 = 2

params = [x_h, x_w, x_c, out_n, f_s1, f_s2, f_s3, f_n1, f_n2, f_n3, a_n1, a_n2, a_n3]

In [0]:
# 모델 만들기
model = tf_Model()
model.model_make(params)

In [0]:
# 모델 불러오기
model_path = 'drive/My Drive/model/'
model = tf_Model()
model.model_make(params)
model.model_load(model_path + 'awd.ckpt')

In [0]:
'''
깃 허브의 오픈소스를 조금 수정한 것
Res Net - 50 : 동작이 검증된 이미지에 대한 CNN 모델
출처: https://github.com/KaihuaTang/ResNet50-Tensorflow-Face-Recognition/blob/master/ResNet.py
'''

import tensorflow as tf
import math

class Res_Model:
    # 클래스 초기화 함수
    def __init__(self):
        self.data_dict = None

        self.var_dict = {}
        self.trainable = True
        self.open_tensorboard = False
        self.is_training = True
        # 세션 준비
        tf.reset_default_graph()
        self.sess = tf.Session()
        
    def res_block_3_layers(self, bottom, channel_list, name, change_dimension = False, block_stride = 1):
        """
        bottom: input values (X)
        channel_list : number of channel in 3 layers
        name: block name
        """
        if (change_dimension):
            short_cut_conv = self.conv_layer(bottom, 1, bottom.get_shape().as_list()[-1], channel_list[2], block_stride, name + "_ShortcutConv")
            block_conv_input = self.batch_norm(short_cut_conv)
        else:
            block_conv_input = bottom

        block_conv_1 = self.conv_layer(bottom, 1, bottom.get_shape().as_list()[-1], channel_list[0], block_stride, name + "_lovalConv1")
        block_norm_1 = self.batch_norm(block_conv_1)
        block_relu_1 = tf.nn.relu(block_norm_1)

        block_conv_2 = self.conv_layer(block_relu_1, 3, channel_list[0], channel_list[1], 1, name + "_lovalConv2")
        block_norm_2 = self.batch_norm(block_conv_2)
        block_relu_2 = tf.nn.relu(block_norm_2)

        block_conv_3 = self.conv_layer(block_relu_2, 1, channel_list[1], channel_list[2], 1, name + "_lovalConv3")
        block_norm_3 = self.batch_norm(block_conv_3)
        block_res = tf.add(block_conv_input, block_norm_3)
        relu = tf.nn.relu(block_res)

        return relu

    # 배치 정규화
    def batch_norm(self, inputsTensor):
        """
        Batchnorm
        """
        _BATCH_NORM_DECAY = 0.99
        _BATCH_NORM_EPSILON = 1e-12
        return tf.layers.batch_normalization(inputs=inputsTensor, axis = 3, momentum=_BATCH_NORM_DECAY, epsilon=_BATCH_NORM_EPSILON, center=True, scale=True, training=self.is_training)

    # 평균값 풀링
    def avg_pool(self, bottom, kernal_size = 2, stride = 2, name = "avg"):
        """
        bottom: input values (X)
        kernal_size : n * n kernal
        stride : stride
        name : block_layer name
        """
        return tf.nn.avg_pool(bottom, ksize=[1, kernal_size, kernal_size, 1], strides=[1, stride, stride, 1], padding='VALID', name=name)

    # 최대값 풀링
    def max_pool(self, bottom, kernal_size = 2, stride = 2, name = "max"):
        """
        bottom: input values (X)
        kernal_size : n * n kernal
        stride : stride
        name : block_layer name
        """
        return tf.nn.max_pool(bottom, ksize=[1, kernal_size, kernal_size, 1], strides=[1, stride, stride, 1], padding='SAME', name=name)

    # 컨볼루션계층
    def conv_layer(self, bottom, kernal_size, in_channels, out_channels, stride, name):
        """
        bottom: input values (X)
        kernal_size : n * n kernal
        in_channels: number of input filters
        out_channels : number of output filters
        stride : stride
        name : block_layer name
        """
        with tf.variable_scope(name):
            filt, conv_biases = self.get_conv_var(kernal_size, in_channels, out_channels, name)

            conv = tf.nn.conv2d(bottom, filt, [1,stride,stride,1], padding='SAME')
            bias = tf.nn.bias_add(conv, conv_biases)

            tf.summary.histogram('weight', filt)
            tf.summary.histogram('bias', conv_biases)

            return bias

    # 완전연결계층
    def fc_layer(self, bottom, in_size, out_size, name):
        """
        bottom: input values (X)
        in_size : number of input feature size
        out_size : number of output feature size
        """
        with tf.variable_scope(name):
            weights, biases = self.get_fc_var(in_size, out_size, name)

            x = tf.reshape(bottom, [-1, in_size])
            fc = tf.nn.bias_add(tf.matmul(x, weights), biases)

            tf.summary.histogram('weight', weights)
            tf.summary.histogram('bias', biases)

            return fc

    # 컨볼루션 계층에 필요한 변수 계산, 반환
    def get_conv_var(self, filter_size, in_channels, out_channels, name):
        """
        filter_size : 3 * 3
        in_channels : number of input filters
        out_channels : number of output filters
        name : block_layer name
        """
        initial_value = tf.truncated_normal([filter_size, filter_size, in_channels, out_channels], 0.0, stddev = 1 / math.sqrt(float(filter_size * filter_size)))
        filters = self.get_var(initial_value, name, 0, name + "_filters")

        initial_value = tf.truncated_normal([out_channels], 0.0, 1.0)
        biases = self.get_var(initial_value, name, 1, name + "_biases")

        return filters, biases

    # 완전연결 계층에 필요한 변수 계산, 반환
    def get_fc_var(self, in_size, out_size, name):
        """
        in_size : number of input feature size
        out_size : number of output feature size
        name : block_layer name
        """
        initial_value = tf.truncated_normal([in_size, out_size], 0.0, stddev = 1 / math.sqrt(float(in_size)))
        weights = self.get_var(initial_value, name, 0, name + "_weights")

        initial_value = tf.truncated_normal([out_size], 0.0, 1.0)
        biases = self.get_var(initial_value, name, 1, name + "_biases")

        return weights, biases


    def get_var(self, initial_value, name, idx, var_name):
        """
        load variables from Loaded model or new generated random variables
        initial_value : random initialized value
        name: block_layer name
        index: 0,1 weight or bias
        var_name: name + "_filter"/"_bias"
        """
        if((name, idx) in self.var_dict):
            print("Reuse Parameters...")
            print(self.var_dict[(name, idx)])
            return self.var_dict[(name, idx)]

        if self.data_dict is not None and name in self.data_dict:
            value = self.data_dict[name][idx]
        else:
            value = initial_value

        if self.trainable:
            var = tf.Variable(value, name=var_name)
        else:
            var = tf.constant(value, dtype=tf.float32, name=var_name)

        self.var_dict[(name, idx)] = var

        # print var_name, var.get_shape().as_list()
        assert var.get_shape() == initial_value.get_shape()

        return var
        
    def model_make(self):
        """
        load variable from npy to build the Resnet or Generate a new one
        :param rgb: rgb image [batch, height, width, 3] values scaled [0, 1]
        """
        self.x = tf.placeholder(tf.float32, shape = [None, 224, 224, 3])
        self.y = tf.placeholder(tf.int64, shape = [None])
        
        # Preprocessing: Turning RGB to BGR - Mean.
        BGR_MEAN = [104.7546, 124.328, 167.1754]
        red, green, blue = tf.split(axis=3, num_or_size_splits=3, value=self.x)
        assert red.get_shape().as_list()[1:] == [224, 224, 1]
        assert green.get_shape().as_list()[1:] == [224, 224, 1]
        assert blue.get_shape().as_list()[1:] == [224, 224, 1]
        bgr = tf.concat(axis=3, values=[
            blue - BGR_MEAN[0],
            green - BGR_MEAN[1],
            red - BGR_MEAN[2],
        ])
        assert bgr.get_shape().as_list()[1:] == [224, 224, 3]

        self.conv1 = self.conv_layer(bgr, 7, 3, 64, 2, "conv1")
        self.conv_norm_1 = self.batch_norm(self.conv1)
        self.conv1_relu = tf.nn.relu(self.conv_norm_1)

        self.pool1 = self.max_pool(self.conv1_relu, 3, 2, "pool1")
        self.block1_1 = self.res_block_3_layers(self.pool1, [64, 64, 256], "block1_1", True)
        self.block1_2 = self.res_block_3_layers(self.block1_1, [64, 64, 256], "block1_2")
        self.block1_3 = self.res_block_3_layers(self.block1_2, [64, 64, 256], "block1_3")

        self.block2_1 = self.res_block_3_layers(self.block1_3, [128, 128, 512], "block2_1", True, 2)
        self.block2_2 = self.res_block_3_layers(self.block2_1, [128, 128, 512], "block2_2")
        self.block2_3 = self.res_block_3_layers(self.block2_2, [128, 128, 512], "block2_3")
        self.block2_4 = self.res_block_3_layers(self.block2_3, [128, 128, 512], "block2_4")

        self.block3_1 = self.res_block_3_layers(self.block2_4, [256, 256, 1024], "block3_1", True, 2)
        self.block3_2 = self.res_block_3_layers(self.block3_1, [256, 256, 1024], "block3_2")
        self.block3_3 = self.res_block_3_layers(self.block3_2, [256, 256, 1024], "block3_3")
        self.block3_4 = self.res_block_3_layers(self.block3_3, [256, 256, 1024], "block3_4")
        self.block3_5 = self.res_block_3_layers(self.block3_4, [256, 256, 1024], "block3_5")
        self.block3_6 = self.res_block_3_layers(self.block3_5, [256, 256, 1024], "block3_6")

        self.block4_1 = self.res_block_3_layers(self.block3_6, [512, 512, 2048], "block4_1", True, 2)
        self.block4_2 = self.res_block_3_layers(self.block4_1, [512, 512, 2048], "block4_2")
        self.block4_3 = self.res_block_3_layers(self.block4_2, [512, 512, 2048], "block4_3")

        self.pool2 = self.avg_pool(self.block4_3, 7, 1, "pool2")

        self.fc1 = self.fc_layer(self.pool2, 2048, 2, "fc2")
        self.fc2 = self.fc_layer(self.fc1, 2, 5, "fc5")

        self.loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = self.fc2, labels = self.y)
        self.cost = tf.reduce_sum(self.loss)

        self.softmax = tf.nn.softmax(self.fc2)
        self.arg_m   = tf.argmax(self.softmax,1)
        self.correct_prediction = tf.equal(self.arg_m, self.y)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        self.train = tf.train.MomentumOptimizer(1e-05, 0.9).minimize(self.cost)

        self.prob = tf.nn.softmax(self.fc2, name="prob")
                
        self.sess.run(tf.global_variables_initializer())
            
    def model_save(self, path):
        saver = tf.train.Saver()
        save_path = saver.save(self.sess, path)
        print("SavePath: %s"%(save_path))
        
    def model_load(self, path):
        saver = tf.train.Saver()
        saver.restore(self.sess, path)
        
    def model_train(self, x_b, y_b):
        self.sess.run(self.train,feed_dict={self.x: x_b, self.y: y_b})

    def get_accuracy(self, x_b, y_b):
        res = self.sess.run(self.accuracy,feed_dict={self.x: x_b, self.y: y_b})
        return res
    
    def get_xy(self, x_b):
        res = self.sess.run(self.fc1,feed_dict={self.x: x_b})
        res2 = self.sess.run(self.arg_m,feed_dict={self.x: x_b})
        
        return res, res2
    
    def print_inf(self, x_b):
        res = self.sess.run(self.fc1,feed_dict={self.x: x_b})
        print(res)
        res = self.sess.run(self.softmax,feed_dict={self.x: x_b})        
        print(res)
        res = self.sess.run(self.arg_m,feed_dict={self.x: x_b})
        print(res)
        print()

In [0]:
# Res를 위한 모델 만들기
model = Res_Model()
model.model_make()

In [0]:
# Res 모델 불러오기
model_path = 'drive/My Drive/AWD/'
model = Res_Model()
model.model_make()
model.model_load(model_path + 'res.ckpt')

W0811 13:58:09.804135 140679071790976 deprecation.py:323] From <ipython-input-2-2d41771c0b9a>:55: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
W0811 13:58:17.795596 140679071790976 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [0]:
# tf_Model 모델 학습을 위한 준비
import numpy as np
import cv2


# 라벨 데이터 준비 - 100개, ont_hot 인코딩

t_list = np.zeros((100,5))

val = 0
cnt = 0
for i in range(100):
    t_list[i][val] = 1
    
    cnt+=1
    if cnt == 20:
        cnt=0
        val+=1

print(t_list.shape)


(100, 5)
(100, 224, 224, 3)


In [0]:
# Res 모델 학습을 위한 라벨 준비(전체 데이터)
import numpy as np
import pickle

cls_path = 'drive/My Drive/clusters.pickle'

data = None

with open(cls_path, 'rb') as f:
    data = pickle.load(f)
    
t_list = np.zeros((2323))

for key in data.keys():
    for name in data[key]:
        t_list[name - 1] = key

t_list = t_list.astype(np.int32)
print("numpy dtype: ", t_list.dtype)

print("t_list shape: ", t_list.shape)
print(t_list)


numpy dtype:  int32
t_list shape:  (2323,)
[0 3 4 ... 2 3 4]


In [0]:
# 전체 이미지 데이터 준비
import cv2
import numpy as np

img_path = 'drive/My Drive/new_gray_png/'

images = []
for i in range(2323):
    full_path = img_path + str(i + 1) + '.png'
    img = cv2.imread(full_path, cv2.IMREAD_UNCHANGED)
    images.append(img)

images = np.array(images)

print(images.shape)

(2323, 224, 224, 3)


In [0]:
# 훈련 이미지 데이터 준비
import cv2
import numpy as np
import pickle

# 분류된 훈련할 데이터 집합 가져오기(사전 자료구조)
T_path = 'drive/My Drive/train.pickle'

data = None
with open(T_path, 'rb') as f:
    data = pickle.load(f)

img_path = 'drive/My Drive/new_gray_png/'

images = []

for k in data.keys():
    for name in data[k]:
        f_p = img_path + str(name) + '.png'
        img = cv2.imread(f_p, cv2.IMREAD_UNCHANGED)
        images.append(img)

images = np.array(images)

print("Images shape", images.shape)

# 5개의 클래스당 20개씩의 훈련데이터 = 100개
t_list = np.zeros((100))

cnt = 0
val = 0
for i in range(100):
    t_list[i] = val
    
    cnt += 1
    if cnt == 20:
        cnt = 0
        val += 1

t_list = t_list.astype(np.int32)
print("numpy dtype: ", t_list.dtype)
print("t_list shape: ", t_list.shape)
print(t_list)

Images shape (100, 224, 224, 3)
numpy dtype:  int32
t_list shape:  (100,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]


In [0]:
# Res 모델을 위한 훈련시키기

print_cnt = 10
repeat_cnt = 5000

data_size = 100
batch_size = 20

b_l = np.random.choice(data_size, data_size, replace=False)

for i in range(repeat_cnt + 1):
    
    if len(b_l) > batch_size:
        data_mask = b_l[:batch_size]
        b_l = b_l[batch_size:]
        
    else:
        t_l = b_l
        b_l = np.random.choice(data_size, data_size, replace=False)

        b_l = np.concatenate((t_l,b_l), axis=None)

        data_mask = b_l[:batch_size]
        b_l = b_l[batch_size:]
    
    x_b = images[data_mask]
    y_b = t_list[data_mask]
    
    
    if i % print_cnt == 0:
        
        train_accuracy = model.get_accuracy(x_b, y_b)
        print("step %d, training accuracy %.4f"%(i, train_accuracy))
        
        
        xy, am = model.get_xy(x_b)
        print("pred: ", am)
        print("ans : ", y_b)
        
        
    
    model.model_train(x_b, y_b)

print("End Of Program-Train")

# 모델을 저장
model_path = 'drive/My Drive/AWD/'
model.model_save(model_path + 'res.ckpt')

step 0, training accuracy 0.7500
pred:  [1 2 1 2 3 4 0 2 4 4 0 3 4 3 1 3 2 1 2 1]
ans :  [1 2 1 0 0 3 0 2 4 4 0 0 4 3 1 0 2 1 2 1]
step 10, training accuracy 0.9000
pred:  [2 3 1 4 1 1 3 0 2 0 3 2 4 3 4 0 1 2 4 3]
ans :  [4 3 1 4 1 1 3 3 2 0 3 2 4 3 4 0 1 2 4 3]
step 20, training accuracy 0.8000
pred:  [3 1 4 2 0 4 2 1 2 4 4 1 3 4 3 3 2 4 0 1]
ans :  [3 2 4 4 0 4 2 1 2 4 4 0 4 4 3 3 2 4 0 1]
step 30, training accuracy 0.5500
pred:  [3 4 0 3 4 0 2 4 1 2 0 3 4 3 1 2 1 0 0 2]
ans :  [0 4 0 0 2 0 2 2 1 1 2 3 3 3 1 3 1 0 0 4]
step 40, training accuracy 0.8500
pred:  [1 0 1 4 4 4 0 4 1 1 3 4 4 2 2 3 3 4 0 1]
ans :  [1 0 2 4 4 4 0 4 1 1 3 2 4 2 2 3 3 4 3 1]
step 50, training accuracy 0.9000
pred:  [3 1 1 4 4 3 1 3 2 4 1 2 4 4 0 0 3 4 0 0]
ans :  [2 1 1 4 4 3 1 3 1 4 1 2 4 4 0 0 3 4 0 0]
step 60, training accuracy 0.7000
pred:  [1 4 1 1 1 1 4 0 3 0 4 4 1 3 1 4 0 3 4 0]
ans :  [1 2 1 1 1 1 4 0 2 0 4 2 1 3 1 4 3 0 2 0]
step 70, training accuracy 0.9500
pred:  [2 1 4 1 1 1 4 0 3 1 4 3 0 3 1 1 3 4

In [0]:
# 모든 데이터에 대해 xy 좌표값 계산하기
xy_list = []
argm_list = []
cnt = 0

bef = 0
aft = 20
for i in range(116):
    xy, am = model.get_xy(images[bef:aft])
    
    for i in range(len(xy)):
        xy_list.append(xy[i])
        argm_list.append(am[i])
    
    bef = aft
    aft += 20
    
xy, am = model.get_xy(images[bef:2323])
for i in range(len(xy)):
    xy_list.append(xy[i])
    argm_list.append(am[i])


print(xy_list)
print(argm_list)


[array([-2.0319548, 13.368402 ], dtype=float32), array([ 4.5675993, -0.2152335], dtype=float32), array([ 6.9440336, -3.8514557], dtype=float32), array([-5.341563 ,  5.7516336], dtype=float32), array([-2.2859292,  3.3337543], dtype=float32), array([-0.80394447, -2.634318  ], dtype=float32), array([-0.13491404,  2.9426222 ], dtype=float32), array([4.5314283 , 0.20819587], dtype=float32), array([ 3.827728 , -1.9811578], dtype=float32), array([ 7.414772 , -2.5482676], dtype=float32), array([ 3.6467707, -5.3131104], dtype=float32), array([ 3.4961827, -4.441802 ], dtype=float32), array([ 4.0637035 , -0.46021897], dtype=float32), array([1.1927269, 2.1048985], dtype=float32), array([-0.82043564,  3.2437072 ], dtype=float32), array([ 1.3370295, -3.9365597], dtype=float32), array([0.51790047, 0.02098447], dtype=float32), array([ 3.7661984, -3.8240457], dtype=float32), array([-2.448271  ,  0.27232057], dtype=float32), array([-0.22112948, -0.97984105], dtype=float32), array([ 0.34421575, -1.192493

In [0]:
no_list = {}

no_list[0] = 0
no_list[1] = 0
no_list[2] = 0
no_list[3] = 0
no_list[4] = 0

for item in argm_list:
  no_list[item] = no_list[item] + 1

print(no_list)

{0: 408, 1: 389, 2: 467, 3: 426, 4: 633}


In [0]:
# 모든 데이터의 xy 좌표값 저장
import pickle

with open('drive/My Drive/AWD/xylist.pk', 'wb') as f:
    pickle.dump(xy_list, f, pickle.HIGHEST_PROTOCOL)

In [2]:
# 모든 데이터의 xy 좌표값 가져오기
import pickle
with open('drive/My Drive/AWD/xylist.pk', 'rb') as f:
    xy_list = pickle.load(f)

print(len(xy_list))

2323


In [0]:
'''
xy 좌표를 x와 y를 따로 나누기
'''
import numpy as np

n_list = []
for i in range(len(xy_list)):
    n_list.append(xy_list[i].tolist())
    
n_list=np.array(n_list)

x_l = []
y_l = []

for data in n_list:
    x_l.append(data[0])
    y_l.append(data[1])

x_l = np.array(x_l)
y_l = np.array(y_l)

In [0]:
'''
x좌표와 y좌표가 일정한 범위 내에 있도록 범위제한
'''
x_l = x_l - min(x_l)
y_l = y_l - min(y_l)

x_l = (x_l/max(x_l))*10000-5000
y_l = (y_l/max(y_l))*10000-5000

print('min_x: ', min(x_l))
print('max_x: ', max(x_l))
print('mean_x: ', sum(x_l)/len(x_l))
print()
print('min_y: ', min(y_l))
print('max_y: ', max(y_l))
print('mean_y: ', sum(y_l)/len(y_l))

min_x:  -5000.0
max_x:  5000.0
mean_x:  1341.750756307506

min_y:  -5000.0
max_y:  5000.0
mean_y:  -2613.630332677397


In [0]:
'''
K-Means를 위한 입력데이터 옮기기
'''

# 입력 데이터
X = np.zeros((len(x_l), 2))

for i in range(len(x_l)):
    X[i,0]=x_l[i]
    X[i,1]=y_l[i]

In [0]:
'''
입력 데이터 그려보기
'''
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

df = pd.DataFrame({"x": [v[0] for v in X],
                   "y": [v[1] for v in X]})
sns.lmplot("x", "y", data=df, fit_reg=False, size=6)
plt.show()

In [6]:
'''
K-means 구현
5개의 군집으로 그룹화
'''

import tensorflow as tf
import pandas as pd
import seaborn as sns

tf.reset_default_graph()

# 모든 데이터를 상수 텐서로 옮김
vectors = tf.constant(X.tolist())

# 초기 단계 : 중심 k(4)개를 입력데이터에서 무작위로 선택
k = 5
centroides = tf.Variable(tf.slice(tf.random_shuffle(vectors),[0,0],[k,-1]))
  
expanded_vectors = tf.expand_dims(vectors, 0)
expanded_centroides = tf.expand_dims(centroides, 1)

# 할당 단계 : 유클리드 제곱거리 사용
diff = expanded_vectors - expanded_centroides
sqr = tf.square(diff)
distances = tf.reduce_sum(sqr, 2)
assignments = tf.argmin(distances,0)

# 업데이트 : 새로운 중심 계산
conc_list = []
for c in range(k):    
    equal_s = tf.equal(assignments, c)
    where_s = tf.where(equal_s)
    reshape_s = tf.reshape(where_s,[1, -1])
    gather_s = tf.gather(vectors,reshape_s)
    reduce_s = tf.reduce_mean(gather_s, reduction_indices=[1])
    conc_list.append(reduce_s)

means = tf.concat(conc_list, 0)

update_centroides = tf.assign(centroides, means)

init_op = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init_op)

for step in range(100):
    _, centroid_values, assignment_values = sess.run([update_centroides, centroides, assignments])


# assignment_values 텐서의 결과를 확인

data = {"x": [], "y": [], "cluster": []}

for i in range(len(assignment_values)):
    data["x"].append(X[i][0])
    data["y"].append(X[i][1])
    data["cluster"].append(assignment_values[i])

'''
df = pd.DataFrame(data)
sns.lmplot("x", "y", data=df, fit_reg=False, size=6, hue="cluster", legend=False)
plt.show()
'''

W0812 03:57:28.328675 140514493831040 deprecation.py:323] From <ipython-input-6-6b1890c92479>:32: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0812 03:57:28.366716 140514493831040 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf_should_use.py:193: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


'\ndf = pd.DataFrame(data)\nsns.lmplot("x", "y", data=df, fit_reg=False, size=6, hue="cluster", legend=False)\nplt.show()\n'

In [8]:
# 위 군집화의 결과를 클러스터 사전 데이터로 바꾸기
clusters={}
for i in range(5):
    clusters[i]= []

for i in range(len(assignment_values)):
    clusters[assignment_values[i]].append(i)
    
for i in range(5):
    clusters[i].sort()

for i in range(5):
    print('{} : {}'.format(i, clusters[i]))
    
print()

for i in range(5):
    print('{} 번 클래스인 탬플릿 {} 개'.format(i, len(clusters[i])))


0 : [1, 2, 7, 8, 9, 10, 11, 12, 17, 22, 27, 32, 34, 35, 36, 44, 45, 47, 54, 56, 57, 58, 59, 65, 66, 67, 69, 72, 75, 78, 79, 82, 85, 86, 90, 94, 96, 97, 99, 100, 102, 103, 107, 110, 111, 116, 118, 119, 120, 121, 125, 127, 128, 129, 130, 132, 133, 136, 140, 143, 145, 147, 149, 153, 154, 157, 160, 161, 165, 172, 176, 177, 179, 182, 186, 188, 189, 190, 195, 196, 199, 203, 204, 209, 211, 214, 216, 219, 224, 228, 231, 232, 237, 238, 239, 240, 241, 243, 245, 246, 248, 261, 262, 267, 268, 271, 272, 274, 276, 280, 282, 286, 287, 290, 292, 294, 295, 298, 301, 311, 312, 316, 319, 321, 327, 328, 332, 333, 336, 337, 339, 340, 341, 342, 343, 344, 346, 349, 359, 364, 365, 368, 370, 371, 374, 377, 381, 383, 384, 386, 389, 392, 393, 394, 395, 398, 400, 411, 412, 416, 417, 419, 420, 421, 426, 427, 434, 435, 436, 440, 442, 448, 449, 451, 455, 457, 460, 467, 468, 470, 479, 485, 491, 493, 496, 499, 502, 505, 510, 511, 513, 514, 516, 518, 519, 520, 522, 523, 528, 531, 532, 534, 538, 539, 547, 550, 555, 556,

In [0]:
# 저장하기  
with open('drive/My Drive/AWD/clusters.pickle', 'wb') as f:
    pickle.dump(clusters, f, pickle.HIGHEST_PROTOCOL)
